In [55]:
import pandas as pd
from bs4 import BeautifulSoup
import os
import numpy as np

pd.set_option('display.max_rows', 250)
pd.set_option('display.max_columns', None)

DATA_DIR = "data/player_stats"
links = os.listdir(DATA_DIR)
links = [os.path.join(DATA_DIR, f) for f in links if f.endswith(".html")]

In [56]:
def parse_html(box_score):
    with open(box_score) as f:
        html = f.read()
    
    soup = BeautifulSoup(html)
    
    [s.decompose() for s in soup.select("tr.thead")]
    return soup

In [57]:
def read_player_name(soup):
    parent_div = soup.find(id="bottom_nav_container")
    if not parent_div:
        return None

    first_p_tag = parent_div.find('p')
    nested_u_tag = first_p_tag.find('u')

    text = nested_u_tag.get_text()
    
    text_split = text.split(" ")
    name = ""
    for i in range(len(text_split) - 1):
        name += (text_split[i] + " ")
    
    return name[:-1]

In [58]:
def read_table(soup):
    df = pd.read_html(str(soup), attrs={"id": "pgl_basic"})[0]
    return df

In [59]:
def convert_minutes(val):
    mp_old = val.split(':')
    minutes, secs = int(mp_old[0]), int(mp_old[1])
    return (minutes + (secs / 60))

def convert_age(val):
    age_old = val.split('-')
    years, days = int(age_old[0]), int(age_old[1])
    return (years + (days / 365))

In [60]:
box_scores = []

for link in links:
    soup = parse_html(link)
    name = read_player_name(soup)
    if not name:
        print("Skipping", link, "due to empty file")
        continue
    
    df = read_table(soup)
    
    #Handle cases where +/- is not recorded
    if '+/-' not in df.columns:
        df['+/-'] = np.nan
    
    df.dropna(subset=['G', 'MP'], inplace=True)
    df.drop(columns=['Rk'], inplace=True)

    df = df.rename(columns={"Unnamed: 5" : "Away?", "Unnamed: 7": "Result"})

    df['Away?'] = df['Away?'].fillna(False).astype(bool)
    df['Result'] = pd.to_numeric(df['Result'].str.replace(r'[WL()]', '', regex=True))
    df['Date'] = pd.to_datetime(df['Date'])

    df['MP'] = df['MP'].apply(convert_minutes)
    df['Age'] = df['Age'].apply(convert_age)

    numeric_columns = ['GS', 'FG','FGA', 'FG%', '3P', '3PA', '3P%', 'FT', 'FTA', 'FT%', 'ORB', 'DRB','TRB', 
                       'AST', 'STL', 'BLK', 'TOV', 'PF', 'PTS', 'GmSc', '+/-' ]

    for col in numeric_columns:
        df[col] = pd.to_numeric(df[col])
    
    player_id = link.split('/')[2].split('_')[0]
    
    df.insert(0, 'Player ID', player_id)
    df.insert(1, 'Name', name)
    
    box_scores.append(df)
    
    if len(box_scores) % 250 == 0:
        print(f"{len(box_scores)} / {len(links)}")

250 / 19349
500 / 19349
750 / 19349
1000 / 19349
1250 / 19349
1500 / 19349
1750 / 19349
2000 / 19349
2250 / 19349
2500 / 19349
2750 / 19349
3000 / 19349
3250 / 19349
3500 / 19349
3750 / 19349
4000 / 19349
4250 / 19349
4500 / 19349
4750 / 19349
5000 / 19349
5250 / 19349
5500 / 19349
5750 / 19349
6000 / 19349
6250 / 19349
6500 / 19349
6750 / 19349
7000 / 19349
7250 / 19349
7500 / 19349
7750 / 19349
8000 / 19349
8250 / 19349
8500 / 19349
8750 / 19349
9000 / 19349
9250 / 19349
9500 / 19349
9750 / 19349
10000 / 19349
10250 / 19349
10500 / 19349
10750 / 19349
11000 / 19349
11250 / 19349
Skipping None due to empty file
11500 / 19349
11750 / 19349
12000 / 19349
Skipping None due to empty file
12250 / 19349
12500 / 19349
12750 / 19349
13000 / 19349
13250 / 19349
13500 / 19349
13750 / 19349
14000 / 19349
14250 / 19349
14500 / 19349
14750 / 19349
15000 / 19349
15250 / 19349
15500 / 19349
15750 / 19349
16000 / 19349
16250 / 19349
16500 / 19349
16750 / 19349
17000 / 19349
17250 / 19349
17500 / 1934

In [61]:
df_players = pd.concat(box_scores, ignore_index=True)

In [62]:
df_players

,Player ID,Name,G,Date,Age,Tm,Away?,Opp,Result,GS,MP,FG,FGA,FG%,3P,3PA,3P%,FT,FTA,FT%,ORB,DRB,TRB,AST,STL,BLK,TOV,PF,PTS,GmSc,+/-
0,martike02,Kevin Martin,1.0,2006-11-01,23.747945,SAC,True,MIN,-9,1.0,38.450000,9,15.0,0.600,1,3.0,0.333,4,6,0.667,3.0,2.0,5.0,0.0,2.0,0.0,5.0,2.0,23,14.2,-16.0
1,martike02,Kevin Martin,2.0,2006-11-03,23.753425,SAC,True,CHI,1,1.0,44.983333,10,15.0,0.667,1,2.0,0.500,9,10,0.900,3.0,2.0,5.0,0.0,3.0,0.0,1.0,1.0,30,27.4,5.0
2,martike02,Kevin Martin,3.0,2006-11-04,23.756164,SAC,True,MIL,-13,1.0,32.500000,5,14.0,0.357,1,4.0,0.250,4,4,1.000,2.0,1.0,3.0,2.0,1.0,0.0,1.0,1.0,15,9.9,-25.0
3,martike02,Kevin Martin,4.0,2006-11-06,23.761644,SAC,False,MIN,12,1.0,39.566667,5,12.0,0.417,1,5.0,0.200,8,8,1.000,0.0,4.0,4.0,5.0,1.0,0.0,2.0,2.0,19,15.5,8.0
4,martike02,Kevin Martin,5.0,2006-11-08,23.767123,SAC,False,DET,13,1.0,36.683333,10,16.0,0.625,2,4.0,0.500,8,8,1.000,2.0,4.0,6.0,2.0,2.0,0.0,3.0,2.0,30,25.0,15.0
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
1031798,amundlo01,Lou Amundson,56.0,2012-04-17,29.361644,IND,True,PHI,5,0.0,10.550000,0,2.0,0.000,0,0.0,NaN,0,0,NaN,1.0,1.0,2.0,1.0,0.0,0.0,0.0,2.0,0,-0.5,3.0
1031799,amundlo01,Lou Amundson,57.0,2012-04-19,29.367123,IND,False,MIL,9,0.0,22.750000,2,7.0,0.286,0,0.0,NaN,0,0,NaN,3.0,1.0,4.0,0.0,1.0,1.0,0.0,1.0,4,3.6,-7.0
1031800,amundlo01,Lou Amundson,58.0,2012-04-21,29.372603,IND,False,PHI,-3,0.0,12.116667,2,3.0,0.667,0,0.0,NaN,0,0,NaN,2.0,0.0,2.0,1.0,2.0,0.0,0.0,4.0,4,5.2,-1.0
1031801,amundlo01,Lou Amundson,59.0,2012-04-23,29.378082,IND,False,DET,6,0.0,15.950000,1,5.0,0.200,0,0.0,NaN,0,4,0.000,5.0,0.0,5.0,0.0,1.0,1.0,0.0,2.0,2,1.7,-1.0


In [63]:
df_players.to_csv("player_box_scores.csv")

In [64]:
df_players.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 1031803 entries, 0 to 1031802
Data columns (total 31 columns):
 #   Column     Non-Null Count    Dtype         
---  ------     --------------    -----         
 0   Player ID  1031803 non-null  object        
 1   Name       1031803 non-null  object        
 2   G          1031803 non-null  float64       
 3   Date       1031803 non-null  datetime64[ns]
 4   Age        1031803 non-null  float64       
 5   Tm         1031803 non-null  object        
 6   Away?      1031803 non-null  bool          
 7   Opp        1031803 non-null  object        
 8   Result     1031803 non-null  int64         
 9   GS         1031802 non-null  float64       
 10  MP         1031803 non-null  float64       
 11  FG         1031803 non-null  int64         
 12  FGA        1031798 non-null  float64       
 13  FG%        981847 non-null   float64       
 14  3P         1031803 non-null  int64         
 15  3PA        1027652 non-null  float64       
 16  